In [103]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [4]:
df =  pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2019')

In [8]:
df.head(1)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Амурская область,Unnamed: 5,Unnamed: 6,Unnamed: 7,Астраханская область,Unnamed: 9,...,Unnamed: 51,Томская область,Unnamed: 53,Unnamed: 54,Unnamed: 55,Тюменская область,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,№ показателя,Наименование индикатора/показателя,№ строки,Единица измерения,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 202...,Фактическое значение нарастающим итогом за 202...,Фактическое значение показателя с нарастающим ...,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 202...,...,Фактическое значение показателя с нарастающим ...,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 202...,Фактическое значение нарастающим итогом за 202...,Фактическое значение показателя с нарастающим ...,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 202...,Фактическое значение нарастающим итогом за 202...,Фактическое значение показателя с нарастающим ...,NaN


In [41]:
# получаем список регионов
lst_regions = [region for region in df.columns if 'Unnamed' not in region]

lst_regions


['Амурская область',
 'Астраханская область',
 'Волгоградская область',
 'Воронежская область',
 'Иркутская область',
 'Кемеровская область',
 'Московская область',
 'Новгородская область',
 'РСО-Алания',
 'Республика Татарстан',
 'Свердловская область',
 'Тверская область',
 'Томская область',
 'Тюменская область']

In [73]:
te_df = pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2019',skiprows=1)

In [74]:
te_df.drop(0,axis=0,inplace=True)

In [75]:
te_df.drop(columns=['№ показателя','Единица измерения'],axis=1,inplace=True)

In [76]:
te_df.head(2)

,Наименование индикатора/показателя,№ строки,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 2022 г. (указывается значение только за 4-й квартал!),"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов)",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП,Планируемое значение в 2022 году.1,Фактическое значение по итогам IV квартала 2022 г. (указывается значение только за 4-й квартал!).1,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).1",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.1,...,Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.11,Планируемое значение в 2022 году.12,Фактическое значение по итогам IV квартала 2022 г. (указывается значение только за 4-й квартал!).12,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).12",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.12,Планируемое значение в 2022 году.13,Фактическое значение по итогам IV квартала 2022 г. (указывается значение только за 4-й квартал!).13,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).13",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.13,Unnamed: 60
1,Создана цифровая платформа ЦОПП в сети Интернет,1,да,да,да,да,да,да,да,да,...,да,да,да,да,да,да,да,да,да,NaN
2,"Численность граждан Российской Федерации, охва...",2,4000,2673,9160,14775,3950,3893,7767,49661,...,32440,14542,15957,22864,59376,32000,14214,83853,147044,321862.0


In [77]:
te_df.drop(columns=[column for column in te_df.columns if 'только за 4-й квартал!' in column],inplace=True)

In [78]:
te_df.head(2)

,Наименование индикатора/показателя,№ строки,Планируемое значение в 2022 году,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов)",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП,Планируемое значение в 2022 году.1,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).1",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.1,Планируемое значение в 2022 году.2,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).2",...,Планируемое значение в 2022 году.11,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).11",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.11,Планируемое значение в 2022 году.12,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).12",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.12,Планируемое значение в 2022 году.13,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).13",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.13,Unnamed: 60
1,Создана цифровая платформа ЦОПП в сети Интернет,1,да,да,да,да,да,да,да,да,...,да,да,да,да,да,да,да,да,да,NaN
2,"Численность граждан Российской Федерации, охва...",2,4000,9160,14775,3950,7767,49661,4000,4068,...,4000,10600,32440,14542,22864,59376,32000,83853,147044,321862.0


In [79]:
te_df.drop(columns=['№ строки'],inplace=True)

,Наименование индикатора/показателя,Планируемое значение в 2022 году,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов)",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП,Планируемое значение в 2022 году.1,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).1",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.1,Планируемое значение в 2022 году.2,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).2",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.2,...,Планируемое значение в 2022 году.11,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).11",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.11,Планируемое значение в 2022 году.12,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).12",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.12,Планируемое значение в 2022 году.13,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).13",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.13,Unnamed: 60
1,Создана цифровая платформа ЦОПП в сети Интернет,да,да,да,да,да,да,да,да,да,...,да,да,да,да,да,да,да,да,да,NaN
2,"Численность граждан Российской Федерации, охва...",4000,9160,14775,3950,7767,49661,4000,4068,10171,...,4000,10600,32440,14542,22864,59376,32000,83853,147044,321862.0
3,из них (из строки 02) в возрасте:\n – до 16 лет,3600,8363,13478,2500,3161,12445,2710,2713,NaN,...,3760,9061,27711,4000,3730,10923,22000,62479,121831,193208.0
4,– от 16 до 21 года,300,534,880,400,2030,12455,720,768,NaN,...,180,208,3462,5000,10276,20454,3000,10466,12030,70969.0
5,– от 22 до 35 лет,50,54,124,500,1135,10541,190,190,NaN,...,20,62,504,5000,6223,17528,3000,7206,7945,30578.0


In [94]:
te_df.shape

(56, 44)

In [88]:
lin_df = te_df.transpose()

In [89]:
lin_df.head()

,1,2,3,4,5,6,7,8,9,10,...,47,48,49,50,51,52,53,54,55,56
Наименование индикатора/показателя,Создана цифровая платформа ЦОПП в сети Интернет,"Численность граждан Российской Федерации, охва...",из них (из строки 02) в возрасте:\n – до 16 лет,– от 16 до 21 года,– от 22 до 35 лет,– от 36 до 55 лет,– от 56 до 75 лет,– старше 75 лет,Численность граждан в возрасте от 25 до 65 лет...,"Численность иностранных граждан, охваченных де...",...,– другие ЦОПП,– другие организации (кроме учтенных в строках...,Реализация региональных/федеральных проектов,"Организация, внедрение и реализация в субъекте...",Мониторинг трудоустройства выпускников при над...,Участие в организации чемпионатов «Молодые про...,"Участие в организации чемпионатов ""Абилимпикс""",Участие в федеральных проектах – роль регионал...,Информационное сопровождение деятельности ЦОПП...,Информационное сопровождение деятельности ЦОПП...
Планируемое значение в 2022 году,да,4000,3600,300,50,40,10,0,100,5,...,0,1,нет,0,нет,нет,нет,нет,50,0
"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов)",да,9160,8363,534,54,157,51,1,176,5,...,18,20,нет,0,нет,нет,нет,да,65,0
Фактическое значение показателя с нарастающим итогом от года создания ЦОПП,да,14775,13478,880,124,229,63,1,316,10,...,19,36,Х,Х,Х,Х,Х,Х,Х,Х
Планируемое значение в 2022 году.1,да,3950,2500,400,500,500,48,2,2000,30,...,8,1,да,4,да,да,да,да,15,15


In [90]:
# Утраиваем
lst_triple = [None]
for region in lst_regions:
    for  i in range(3):
        lst_triple.append(region)
lst_triple.append(None)

In [91]:
# Вставляем
lin_df.insert(0,'Наименование субъекта РФ',lst_triple)

In [111]:
# Создаем объект openpyxl
wb = openpyxl.Workbook()


In [112]:
# Получаем список без первых 2 колонок
lst_col = list(lin_df.columns)[2:]

In [113]:
for idx,column in enumerate(lst_col):
    # Создаем маленький датафрейм
    temp_df = lin_df[['Наименование субъекта РФ',column]].transpose()
    # Создаем лист    
    wb.create_sheet(title=str(column),index=idx)
    for row in dataframe_to_rows(temp_df, index=False, header=False):
        wb[str(column)].append(row)
    
    
    

In [114]:
wb.save('Тест.xlsx')

In [97]:
d = lin_df[['Наименование субъекта РФ',2]].transpose()

,Наименование индикатора/показателя,Планируемое значение в 2022 году,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов)",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП,Планируемое значение в 2022 году.1,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).1",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.1,Планируемое значение в 2022 году.2,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).2",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.2,...,Планируемое значение в 2022 году.11,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).11",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.11,Планируемое значение в 2022 году.12,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).12",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.12,Планируемое значение в 2022 году.13,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).13",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.13,Unnamed: 60
Наименование субъекта РФ,None,Амурская область,Амурская область,Амурская область,Астраханская область,Астраханская область,Астраханская область,Волгоградская область,Волгоградская область,Волгоградская область,...,Тверская область,Тверская область,Тверская область,Томская область,Томская область,Томская область,Тюменская область,Тюменская область,Тюменская область,None
2,"Численность граждан Российской Федерации, охва...",4000,9160,14775,3950,7767,49661,4000,4068,10171,...,4000,10600,32440,14542,22864,59376,32000,83853,147044,321862.0


In [93]:
lin_df.to_excel('dtd.xlsx')